In [186]:
import sympy as sp
from sympy import diff, sin, cos, exp, sqrt, Rational

In [168]:
xij, yij, dxij, dyij, rij, θij = sp.symbols('x_{ij} y_{ij} δx_{ij}, δy_{ij}, r_{ij} θ_{ij}')
me, e, t, eps0, pi = sp.symbols('m_e e t ε_0 π')

In this notebook we'll use `sympy` to do some of the heavy lifting in determining the equations of motion. The equations of motion are derived in https://journals.aps.org/prx/supplemental/10.1103/PhysRevX.6.011031/supplement.pdf , but some of the math-heavy steps are missing. Those steps are shown in the following two sections.

# Linearization of $F_x$ for the EOM in $\delta x_i$

A crucial part of the equations of motion is the electron-electron interaction term. For the perturbation in the coordinate $x_i$ we need to linearize the electron-electron interaction force in the x-direction. This force is given by `F_x` and is also written as Eq. (S15) in the link above.

We will use the notation $\delta x_{ij} = \delta x_i - \delta x_j$, $\delta y_{ij} = \delta y_i - \delta y_j$ and $r_{ij}^2 = (x_i - x_j)^2 + (y_i - y_j)^2$.

After setting $x_i \mapsto x_i + \delta x_i$ and $y_i \mapsto y_i + \delta y_i$ we arrive at the following expression for $(x_i - x_j) / |r_i - r_j|^3$

In [252]:
Fx = (rij * cos(θij) + dxij) / (rij ** 3 * (cos(θij) ** 2 * (1 + dxij/(rij * cos(θij))) ** 2 + sin(θij) ** 2 * (1 + dyij/(rij * sin(θij))) ** 2 ) ** (Rational(3,2)))
Fx

(r_{ij}*cos(θ_{ij}) + δx_{ij})/(r_{ij}**3*((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)**(3/2))

To linearize we use the Taylor expansion around the equilibrium coordinates ($\delta x_i = 0, \delta y_i =0$)
$$ 
F_x \approx F_x (\delta x_i = 0, \delta y_i =0) + \frac{\partial F_x}{\partial \delta x_i} (\delta x_i = 0, \delta y_i =0) \delta x_i + \frac{\partial F_x}{\partial \delta y_i} (\delta x_i = 0, \delta y_i =0) \delta y_i
$$

The first term is simply setting $\delta x_i = 0, \delta y_i =0$, which is done with `subs`:

In [192]:
first_term = Fx.subs(dxij, 0).subs(dyij, 0).simplify()
first_term 

cos(θ_{ij})/r_{ij}**2

However, note that the first term won't contribute to the EOM for $\delta x_i$ or $\delta y_i$. For the last two terms we need to differentiate $F_x$ w.r.t. $\delta x_i$ and $\delta y_i$, respectively

In [193]:
linear_term_x = diff(Fx, dxij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_x = linear_term_x.subs(sin(θij)**2, Rational(1, 2) - Rational(1 ,2) * cos(2*θij))
linear_term_x

(-3*cos(2*θ_{ij})/2 - 1/2)/r_{ij}**3

And differentiating with respect to $\delta y_{ij}$ gives

In [194]:
linear_term_y = diff(Fx, dyij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_y

-3*sin(2*θ_{ij})/(2*r_{ij}**3)

To summarize, we've now obtained the electron-electron interaction term $(x_i-x_j) / |r_i - r_j|^3$, which is given by the expression below:

In [208]:
( first_term + linear_term_x * dxij + linear_term_y * dyij ).factor(dxij)

(2*r_{ij}*cos(θ_{ij}) + δx_{ij}*(-3*cos(2*θ_{ij}) - 1) - 3*δy_{ij}*sin(2*θ_{ij}))/(2*r_{ij}**3)

The first term won't contribute to the equations of motion, but the last two terms proportional to $\delta x_{ij}$ and $\delta y_{ij}$ will.

We can start to recognize $k_{ij}^+$ as the term proportional to $\delta x_{ij}$ and similarly $l_{ij}$ as the term proportional to $\delta y_{ij}$.
Note that we omitted $e^2 / 4\pi \varepsilon_0$ and a factor of $1/2$ from double counting. Taking these prefactors into account will give an exact comparison.

# Linearization of $F_y$ for the EOM in $\delta y_i$

The equation of motion for the coordinate $y_i$ contains the term $(y_i - y_j) / |r_i - r_j|^3$. We'll follow a similar procedure as above

In [254]:
Fy = (rij * sin(θij) + dyij) / (rij ** 3 * (cos(θij) ** 2 * (1 + dxij/(rij * cos(θij))) ** 2 + sin(θij) ** 2 * (1 + dyij/(rij * sin(θij))) ** 2 ) ** (Rational(3, 2)))
Fy

(r_{ij}*sin(θ_{ij}) + δy_{ij})/(r_{ij}**3*((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)**(3/2))

To linearize this term we write 
$$ 
F_y \approx F_y (\delta x_i = 0, \delta y_i =0) + \frac{\partial F_y}{\partial \delta x_i} (\delta x_i = 0, \delta y_i =0) \delta x_i + \frac{\partial F_y}{\partial \delta y_i} (\delta x_i = 0, \delta y_i =0) \delta y_i
$$

Again, we start with the term that won't contribute to the equations of motion.

In [255]:
first_term = Fy.subs(dxij, 0).subs(dyij, 0).simplify()
first_term 

sin(θ_{ij})/r_{ij}**2

Here is the term proportional to $\delta x_{ij}$

In [256]:
linear_term_x = diff(Fy, dxij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_x

-3*sin(2*θ_{ij})/(2*r_{ij}**3)

and proportional to $\delta y_{ij}$

In [257]:
linear_term_y = diff(Fy, dyij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_y = linear_term_y.subs(sin(θij)**2, Rational(1, 2) - Rational(1, 2) * cos(2*θij))
linear_term_y

(3*cos(2*θ_{ij})/2 - 1/2)/r_{ij}**3

To summarize: the electron-electron interaction term for the $y$ EOM is given by $(y_i - y_j) / |r_i - r_j|^3$ and the linearized result is

In [215]:
first_term + linear_term_x * dxij + linear_term_y * dyij

sin(θ_{ij})/r_{ij}**2 - 3*δx_{ij}*sin(2*θ_{ij})/(2*r_{ij}**3) + δy_{ij}*(3*cos(2*θ_{ij})/2 - 1/2)/r_{ij}**3

We can start to recognize $k_{ij}^-$ as the term proportional to $\delta y_{ij}$ and similarly $l_{ij}$ as the term proportional to $\delta x_{ij}$.
Note that we omitted $e^2 / 4\pi \varepsilon_0$ and a factor of $1/2$ from double counting. Taking these prefactors into account will give an exact comparison.

# Including the effects of screening

If we include screening, the regular Coulomb potential can be modified to a Yukawa potential

$$
U = \frac{e^{-r/\lambda}}{r}
$$
where $\lambda$ is the interaction cutoff length. We may repeat the previous sections to obtain the versions of $k_{ij}^\pm$ and $l_{ij}$ that include screening. These should be compared to Eqs. C.3 and C.4 in https://schusterlab.stanford.edu/static/pdfs/Koolstra_thesis.pdf Let's get started by figuring out the forces in the $x$ and $y$ direction.

## Force in the $x$ direction

Finding the force requires differentiating the potential. Because $r$ appears in the exponent and also in the denominator, it is not as straightforward. By hand, we would apply the chain rule. `sympy` doesn't care and will do it for you:

In [259]:
rxy = sqrt(xij**2 + yij**2)
lamb = sp.symbols('λ')
xi = sp.symbols('ξ')

U = 1 / rxy * exp(-rxy / lamb)
Fx = diff(U, xij).simplify().subs(sqrt(xij**2 + yij**2), rij)
Fx

x_{ij}*(-r_{ij} - λ)*exp(-r_{ij}/λ)/(r_{ij}**3*λ)

In [260]:
linearized_rij = rij * sqrt(cos(θij) ** 2 * (1 + dxij/(rij * cos(θij))) ** 2 + sin(θij) ** 2 * (1 + dyij/(rij * sin(θij))) ** 2 )
linearized_Fx = Fx.subs(xij, xij + dxij).subs(rij, linearized_rij)
linearized_Fx

(x_{ij} + δx_{ij})*(-r_{ij}*sqrt((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2) - λ)*exp(-r_{ij}*sqrt((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)/λ)/(r_{ij}**3*λ*((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)**(3/2))

The resulting expressions start to grow, but let's not fear yet. Things will look better. 

In [246]:
linearized_Fx = linearized_Fx.subs(xij, rij * cos(θij)).subs(yij, rij * sin(θij)).simplify()
linearized_Fx

-(r_{ij}*sqrt(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2) + λ)*(r_{ij}*cos(θ_{ij}) + δx_{ij})*exp(-r_{ij}*sqrt(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2)/λ)/(r_{ij}**3*λ*(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2)**(3/2))

Things will start to look more compact by using the substitution $\xi = r_{ij} / \lambda$. The first term in the Taylor expansion of the force is:

In [247]:
first_term = linearized_Fx.subs(dxij, 0).subs(dyij, 0).simplify()
first_term.subs(lamb, rij / xi).simplify()

-(ξ + 1)*exp(-ξ)*cos(θ_{ij})/r_{ij}**2

The term that is proportional to $\delta x_{ij}$ is found by differentiating w.r.t. $\delta x_{ij}$:

In [248]:
linear_term_x = diff(linearized_Fx, dxij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_x = linear_term_x.subs(lamb, rij / xi).simplify().subs(cos(θij)**2, Rational(1, 2) + Rational(1, 2) * cos(2*θij))
linear_term_x

(ξ**2*(cos(2*θ_{ij})/2 + 1/2) + ξ*(3*cos(2*θ_{ij})/2 + 1/2) + 3*cos(2*θ_{ij})/2 + 1/2)*exp(-ξ)/r_{ij}**3

The term that is proportional to $\delta y_{ij}$ is found by differentiating w.r.t. $\delta y_{ij}$:

In [249]:
linear_term_y = diff(linearized_Fx, dyij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_y = linear_term_y.subs(lamb, rij / xi).simplify().subs(cos(θij)**2, Rational(1, 2) + Rational(1, 2) * cos(2*θij))
linear_term_y

(ξ**2 + 3*ξ + 3)*exp(-ξ)*sin(2*θ_{ij})/(2*r_{ij}**3)

The final result below should be checked with $k_{ij}^+$ and $l_{ij}$ in C.3 and C.4 in https://schusterlab.stanford.edu/static/pdfs/Koolstra_thesis.pdf You can actually see there is a small error in C4!

In [251]:
(linear_term_x * dxij + linear_term_y * dyij).factor(dxij, dyij)

(δx_{ij}*(ξ**2*cos(2*θ_{ij}) + ξ**2 + 3*ξ*cos(2*θ_{ij}) + ξ + 3*cos(2*θ_{ij}) + 1) + δy_{ij}*(ξ**2*sin(2*θ_{ij}) + 3*ξ*sin(2*θ_{ij}) + 3*sin(2*θ_{ij})))*exp(-ξ)/(2*r_{ij}**3)

## Force in the $y$-direction

You know the drill by know. First differentiate the potential energy (electron-electron interaction) w.r.t. $y_{ij}$ to find $F_y$ that corresponds to the Yukawa potential:

In [227]:
Fy = diff(U, yij).simplify().subs(sqrt(xij**2 + yij**2), rij)
Fy

y_{ij}*(-r_{ij} - λ)*exp(-r_{ij}/λ)/(r_{ij}**3*λ)

Perturb:

In [228]:
linearized_Fy = Fy.subs(yij, yij + dyij).subs(rij, linearized_rij)
linearized_Fy

(y_{ij} + δy_{ij})*(-r_{ij}*sqrt((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2) - λ)*exp(-r_{ij}*sqrt((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)/λ)/(r_{ij}**3*λ*((1 + δx_{ij}/(r_{ij}*cos(θ_{ij})))**2*cos(θ_{ij})**2 + (1 + δy_{ij}/(r_{ij}*sin(θ_{ij})))**2*sin(θ_{ij})**2)**(3/2))

Group and simplify

In [231]:
linearized_Fy = linearized_Fy.subs(xij, rij * cos(θij)).subs(yij, rij * sin(θij)).simplify()
linearized_Fy

-(r_{ij}*sqrt(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2) + λ)*(r_{ij}*sin(θ_{ij}) + δy_{ij})*exp(-r_{ij}*sqrt(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2)/λ)/(r_{ij}**3*λ*(((r_{ij}*sin(θ_{ij}) + δy_{ij})**2 + (r_{ij}*cos(θ_{ij}) + δx_{ij})**2)/r_{ij}**2)**(3/2))

Find the first term in the Taylor series that doesn't affect the EOM:

In [232]:
first_term = linearized_Fy.subs(dxij, 0).subs(dyij, 0).simplify()
first_term.subs(lamb, rij / xi).simplify()

-(ξ + 1)*exp(-ξ)*sin(θ_{ij})/r_{ij}**2

Find the term propoertional to $\delta x_{ij}$:

In [233]:
linear_term_x = diff(linearized_Fy, dxij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_x = linear_term_x.subs(lamb, rij / xi).simplify().subs(cos(θij)**2, Rational(1, 2) + Rational(1, 2) * cos(2*θij))
linear_term_x

(ξ**2 + 3*ξ + 3)*exp(-ξ)*sin(2*θ_{ij})/(2*r_{ij}**3)

Find the term propoertional to $\delta y_{ij}$ (it is actually rather compact given what we started off with)

In [234]:
linear_term_y = diff(linearized_Fy, dyij).subs(dxij, 0).subs(dyij, 0).simplify()
linear_term_y = linear_term_y.subs(lamb, rij / xi).simplify().subs(sin(θij)**2, Rational(1, 2) - Rational(1, 2) * cos(2*θij))
linear_term_y

(ξ**2*(1/2 - cos(2*θ_{ij})/2) + ξ*(1/2 - 3*cos(2*θ_{ij})/2) - 3*cos(2*θ_{ij})/2 + 1/2)*exp(-ξ)/r_{ij}**3

The final result below should be checked with $k_{ij}^+$ and $l_{ij}$ in C.3 and C.4 in https://schusterlab.stanford.edu/static/pdfs/Koolstra_thesis.pdf You can actually see there is a small error in C4!

In [236]:
(linear_term_x * dxij + linear_term_y * dyij).factor(dxij, dyij)

(δx_{ij}*(ξ**2*sin(2*θ_{ij}) + 3*ξ*sin(2*θ_{ij}) + 3*sin(2*θ_{ij})) + δy_{ij}*(-ξ**2*cos(2*θ_{ij}) + ξ**2 - 3*ξ*cos(2*θ_{ij}) + ξ - 3*cos(2*θ_{ij}) + 1))*exp(-ξ)/(2*r_{ij}**3)